# Electrum-FLO target issue

To explain the problem we'll have to first look into how Bitcoin's chunk verification works. 


## Bitcoin's chunk verification  

Electrum BTC wallet upon receving a chunk(ie. 2016 block headers in a serialized form) from the ElectrumX server, sends the whole chunk data for verification to **verify_chunk()**. The first step in the verification process is to calculate the target of the whole chunk. In Bitcoin a new target is calculated after every 2016 blocks, there the target for the 2016 blocks is the same. The following is how the **verify_chunk()** and **get_target()** have been defined 

In [3]:
def verify_chunk(self, index, data):
    num = len(data) // 80  # each block header is 80 bytes long. So num = 2016
    prev_hash = self.get_hash(index * 2016 - 1) 
    target = self.get_target(index-1) # passing index-1 because to get target of header x, you need timestamp of 
                                      # header x-1 and x-1-2016
    
    for i in range(num): # checks locally calc target value with each block header's target value
        raw_header = data[i*80:(i+1) * 80]
        header = deserialize_header(raw_header, index*2016 + i)
        self.verify_header(header, prev_hash, target)
        prev_hash = hash_header(header)

The chunk is broken down into 2016 headers and each of them is verified individually in the for loop 

In [2]:
    def get_target(self, index):
        # compute target from chunk x, used in chunk x+1
        if constants.net.TESTNET:
            return 0
        if index == -1:
            return MAX_TARGET
        if index < len(self.checkpoints):
            h, t = self.checkpoints[index]
            return t
        # new target
        first = self.read_header(index * 2016)
        last = self.read_header(index * 2016 + 2015)
        if not first or not last:
            raise MissingHeader()
        bits = last.get('bits')
        target = self.bits_to_target(bits)
        nActualTimespan = last.get('timestamp') - first.get('timestamp')
        nTargetTimespan = 14 * 24 * 60 * 60
        nActualTimespan = max(nActualTimespan, nTargetTimespan // 4)
        nActualTimespan = min(nActualTimespan, nTargetTimespan * 4)
        new_target = min(MAX_TARGET, (target * nActualTimespan) // nTargetTimespan)
        return new_target

Lets's see the definition of **read_header()** called in the above **get_target()** 

In [ ]:
    def read_header(self, height):
        assert self.parent_id != self.checkpoint
        if height < 0: # Checks if the header to be read is not negative ie. can't exist 
            return
        if height < self.checkpoint:  
            return self.parent().read_header(height)
        if height > self.height(): # checks if the header height is > no of block headers stored locally in the 
            return                 # file blockchain_headers. ie. Failsafe to not skip any header 
        delta = height - self.checkpoint
        name = self.path()
        self.assert_headers_file_available(name)
        with open(name, 'rb') as f:
            f.seek(delta * 80)
            h = f.read(80)
            if len(h) < 80:
                raise Exception('Expected to read a full header. This was only {} bytes'.format(len(h)))
        if h == bytes([0])*80:
            return None
        return deserialize_header(h, height)

Now lets go back to top once again. The following is the function which leads to **verify_chunk()**

In [ ]:
    def connect_chunk(self, idx, hexdata):
        try:
            data = bfh(hexdata)
            self.verify_chunk(idx, data)
            #self.print_error("validated chunk %d" % idx)
            self.save_chunk(idx, data)
            return True
        except BaseException as e:
            self.print_error('verify_chunk %d failed'%idx, str(e))
            return False

In Electrum-BTC, the whole chunk is verified first and then the whole chunk is saved into the local **blockchain_headers** file at once. ie. 2016 headers are first verified and then 2016 headers are saved locally.

This is where the problem with Electrum-FLO wallet comes up. The FLO's retarget times varried at different points in time 90 blocks, 15 blocks & 1 block. But the FLO developers haven't made changes to the wallet's RPC call *get_chunk* as it returns 2016 blocks for the given index. 

## FLO's chunk verification 

The following is the code for FLO's verify chunk. 

In [ ]:
    def verify_chunk(self, index, data):
        num = len(data) // 80
        current_header = (index * 2016)
        #last = (index * 2016 + 2015) 
        prev_hash = self.get_hash(current_header - 1)

        for i in range(num):
            target = self.get_target(current_header -1)
            raw_header = data[i*80:(i+1) * 80]
            header = deserialize_header(raw_header, current_header)
            self.verify_header(header, prev_hash, target)
            prev_hash = hash_header(header)
            current_header = current_header + 1

Since FLO's has varying retarget times but the core wallet's RPC hasn't been changed from Bitcoin. I am making a general assumption that retarget time is 1 block since day 1 and making appropriate changes for the block times inside the **get_target()**. 

Now let us look at FLO's **get_target()**(which is slightly different) 

In [ ]:
    def get_target(self, index):
        if bitcoin.NetworkConstants.TESTNET:
            return 0
        #The range is first 90 blocks because FLO's block time was 90 blocks when it started
        if -1 <= index <= 88:
            return MAX_TARGET
        if index < len(self.checkpoints):
            h, t = self.checkpoints[index]
            return t
        # new target
        headerLast = self.read_header(index)
        height = headerLast["block_height"]

        # check if the height passes is in range for retargeting
        if (height + 1) % self.DifficultyAdjustmentInterval(height + 1) != 0:
            return int(headerLast["bits"])
        .
        . # skipped this part as its irrelavant for our discussion right now
        .

        bnNew = self.target_to_bits(int(bnNew))
        return bnNew

In [ ]:
    def read_header(self, height):
        assert self.parent_id != self.checkpoint
        if height < 0: # Checks if the header to be read is not negative ie. can't exist 
            return
        if height < self.checkpoint:  
            return self.parent().read_header(height)
        if height > self.height(): # checks if the header height is > no of block headers stored locally in the 
            return                 # file blockchain_headers. ie. Failsafe to not skip any header 
        delta = height - self.checkpoint
        name = self.path()
        self.assert_headers_file_available(name)
        with open(name, 'rb') as f:
            f.seek(delta * 80)
            h = f.read(80)
            if len(h) < 80:
                raise Exception('Expected to read a full header. This was only {} bytes'.format(len(h)))
        if h == bytes([0])*80:
            return None
        return deserialize_header(h, height)

**Note - In the above get_target(), index refers to header_height. In BTC's case, index refers to chunk index**

Since for FLO **get_target()** is being called 2016 times per chunk, the condition **if height > self.height()** is False only till the time it height is smaller than the no. of block headers in the local file. 

To solve this problem we need to save each chunk's header immediately after verification.  
